In [1]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_33efb2caf64f4636aedf2ac8b0918c8d"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_9fd5b54c7ca24e46b198257ef2db93a2"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_0121879d12a64bd18af72c6992ca330e"}'
_INPUT4='{"name":"input4","type":0,"uri":"tmp_b3afb39beb1c4e3caef20035420113ea"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_dafc1a848f9642b7bdf482cde3c04c0d"},{"name":"output2","type":0,"uri":"tmp_bbf9885902314f12a2cee82588a90e4f"},{"name":"output3","type":0,"uri":"tmp_9fbb9ca946a04f87aad25ffcc1f93c2f"},{"name":"output4","type":0,"uri":"tmp_85e9ee649ef7493cb1381d4db310099a"}]'

# 自定义参数


In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

import wfio
# from ustciscrLab_A import get_score
from ustciscrBDL_B import get_score

# load data

In [3]:
train_data = wfio.read_dataframe(_INPUT1)
test_data = wfio.read_dataframe(_INPUT2)
train_data_add = wfio.read_dataframe(_INPUT3)
test_data_add = wfio.read_dataframe(_INPUT4)
print(train_data.shape, train_data_add.shape, test_data.shape, test_data_add.shape)
# train_data = pd.concat([train_data, train_data_add], ignore_index=True)
# print(train_data.shape)

test_data = test_data_add

(10000, 114) (10000, 114) (6000, 113) (6000, 113)


# preprocess

In [4]:
cols1 = train_data.columns.tolist()
cols2 = train_data_add.columns.tolist()
cols3 = test_data.columns.tolist()
print(set(cols1)-set(cols2))
print(set(cols2)-set(cols1))
col_dict = {'xyzl': 'hyzl', 'frsjh': 'fdbrxmp', 'cwrysjh': 'bsrxmmp', 'cwryxm': 'bsrxm', 'jdxz': 'xzjd', 'xy': 'hy', 'xydl': 'hydl', 'xyml': 'hyml', 'zczby': 'zczb', 'djkyrq': 'djrq'}

for col1, col2 in col_dict.items():
    train_data.rename(columns={col1: col2},inplace=True)
    
train_data = pd.concat([train_data, train_data_add], ignore_index=True)
print(train_data.shape)

{'xydl', 'xyzl', 'xy', 'cwryxm', 'frsjh', 'jdxz', 'cwrysjh', 'xyml', 'zczby', 'djkyrq'}
{'hyzl', 'xzjd', 'fdbrxmp', 'zczb', 'bsrxmmp', 'hydl', 'hyml', 'djrq', 'hy', 'bsrxm'}
(20000, 114)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [5]:
# desc(train_data)
train_data.isna().any()
train_data.dropna(axis=0,how='all') 
print(train_data.shape)

test_data[test_data=='']=0
test_data[test_data=='""""""""""""""']=0

train_data[train_data=='']=0
train_data[train_data=='""""""""""""""']=0

(20000, 114)


# feature extraction

In [6]:
id_title = 'zjnsrsbh'
label_title = 'yc'
result_title = 'Probability'

def desc(df):
    return list(zip([item for item in df.dtypes.index], [item.name for item in df.dtypes]))

def get_type2index(column):
    type2index = dict()
    for item in column:
        if item not in type2index.keys():
            type2index[item] = len(type2index)
    return type2index

def get_onehot(df, title):
    column = df[title]
    type2index = get_type2index(column)
    onehot_columns = [f"{title}_{index}" for index in type2index.values()]
    dim = len(type2index)
    size = len(column)
    data = np.zeros([size, dim]).astype(np.int32)
    for i, item in enumerate(column):
        data[i][type2index[item]] = 1
    onehot_df = pd.DataFrame(data, columns=onehot_columns, index=column.index)
    return onehot_df

def all_one_or_zero(preds):
    # linear: 1, (0.5, 1 - 1e-5), (0.29, 4e-4), 
    # linear: 2, (0.4 1 - 1e-5), (0.29, 4e-4), 95.43225, no session feature
    # linear: 3, (0.4 1 - 1e-5), (0.3, 4e-4), 95.4124, with session feature
    preds[preds >= 0.4] = 1 - 1e-5
    preds[preds < 0.29] = 4e-4
    #preds[preds >= 0.4] = 1 - 1e-5
    #preds[preds < 0.3] = 4e-4
    return preds

# 最后一次开票季度-注册季度（减2年）
# 离线/在线不同版本（数据日期格式不同）
def get_interval_session(row):
    kps_columns = ['2017q1kps', '2017q2kps', '2017q3kps', '2017q4kps', '2018q1kps', '2018q2kps', '2018q3kps', '2018q4kps', '2019q1kps', '2019q2kps', '2019q3kps', '2019q4kps']
    # 注册季度
    # 离线
    # (datetime.datetime) yyyy/(M)M/(d)d
    # date = row['djkyrq']
    # zc_year, zc_month = date.year, date.month
    # 在线
    # yyyy-MM-dd hh:mm:ss
    zc_year, zc_month, _ = row['djrq'].strip().split(' ')[0].split('-')
    zc_year, zc_month = int(zc_year), int(zc_month)
    zc_year -= 2    # 注册年份减2年
    # 1(3, 4, 5), 2(6, 7, 8), 3(9, 10, 11), 4(12, 1, 2)
    if zc_month <= 2:
        zc_session = 4
        zc_year -= 1
    elif zc_month <= 5:
        zc_session = 1
    elif zc_month <= 8:
        zc_session = 2
    elif zc_month <= 11:
        zc_session = 3
    else:
        zc_session = 4
    # 最后一次开票季度
    for title in reversed(kps_columns):
        if row[title] > 0:
            final_date = title.strip('kps')
            final_year, final_session = final_date.split('q')
            final_year, final_session = int(final_year), int(final_session)
            break
    else:
        # 没有开票记录
        final_year = zc_year
        final_session = zc_session
    interval_session = (final_year - zc_year) * 4 + final_session - zc_session
    return interval_session

# 同一列重复出现的内容
def get_repeat_content(column):
    all_content = set()
    repeat_content = set()
    for item in column:
        if item not in all_content:
            all_content.add(item)
        elif item not in repeat_content:
            repeat_content.add(item)
    return repeat_content

# 地址/手机号码/名称是否重复
def get_repeat_value(cell, repeat_content):
    if cell in repeat_content:
        return 1
    else:
        return 0
    
# 挑选带有标签的特征
def preprocess_label(feature_df, label_df):
    label_df[label_title] = label_df[label_title].astype(np.int32)
    feature_df_with_label = pd.merge(label_df, feature_df, how='inner', on=id_title)
    return feature_df_with_label

def split(feature_df_with_label, train_rate=0.8):
    all_size = len(feature_df_with_label)
    train_size = int(all_size * 0.8)
    test_size = all_size - train_size
    train_selected = np.array([True] * train_size + [False] * test_size)
    np.random.shuffle(train_selected)
    test_selected = ~train_selected
    train_df = feature_df_with_label[train_selected]
    test_df = feature_df_with_label[test_selected]
    return train_df, test_df

In [7]:
def preprocess_feature(feature_df, with_norm=True):
    title_float64 = [
        '2017q1jxje', '2017q1jxsl', '2017q1kpje', '2017q1kpse', '2017q1kpsl', '2017q1rkse',
        '2017q2jxje', '2017q2jxsl', '2017q2kpje', '2017q2kpse', '2017q2kpsl', '2017q2rkse',
        '2017q3jxje', '2017q3jxsl', '2017q3kpje', '2017q3kpse', '2017q3kpsl', '2017q3rkse',
        '2017q4jxje', '2017q4jxsl', '2017q4kpje', '2017q4kpse', '2017q4kpsl', '2017q4rkse',
        '2018q1jxje', '2018q1jxsl', '2018q1kpje', '2018q1kpse', '2018q1kpsl', '2018q1rkse',
        '2018q2jxje', '2018q2jxsl', '2018q2kpje', '2018q2kpse', '2018q2kpsl', '2018q2rkse',
        '2018q3jxje', '2018q3jxsl', '2018q3kpje', '2018q3kpse', '2018q3kpsl', '2018q3rkse',
        '2018q4jxje', '2018q4jxsl', '2018q4kpje', '2018q4kpse', '2018q4kpsl', '2018q4rkse',
        '2019q1jxje', '2019q1jxsl', '2019q1kpje', '2019q1kpse', '2019q1kpsl', '2019q1rkse',
        '2019q2jxje', '2019q2jxsl', '2019q2kpje', '2019q2kpse', '2019q2kpsl', '2019q2rkse',
        '2019q3jxje', '2019q3jxsl', '2019q3kpje', '2019q3kpse', '2019q3kpsl', '2019q3rkse',
        '2019q4jxje', '2019q4jxsl', '2019q4kpje', '2019q4kpse', '2019q4kpsl', '2019q4rkse'
    ]
    title_int64 = [
        '2017q1fphdsl', '2017q1kps', 
        '2017q2fphdsl', '2017q2kps', 
        '2017q3fphdsl', '2017q3kps', 
        '2017q4fphdsl', '2017q4kps', 
        '2018q1fphdsl', '2018q1kps', 
        '2018q2fphdsl', '2018q2kps', 
        '2018q3fphdsl', '2018q3kps', 
        '2018q4fphdsl', '2018q4kps', 
        '2019q1fphdsl', '2019q1kps', 
        '2019q2fphdsl', '2019q2kps', 
        '2019q3fphdsl', '2019q3kps', 
        '2019q4fphdsl', '2019q4kps', 
        'cyrs'
    ]

    title_category = ['hy', 'hydl', 'hyml', 'hyzl'] # 行业，行业大类，行业门类，行业中类
    title_addr = ['scjydz', 'zcdz'] # 生产经营地址地址，注册地址
    title_number = ['bsrxmmp', 'fdbrxmp'] # 财务人员手机号，法人手机号
    title_name = ['bsrxm', 'fddbrxm', 'nsrmc'] # 财务人员姓名，法定代表人姓名，纳税人姓名
    date_col = 'djrq' # 登记开业日期

    feature_titles = list()
    # 无预处理
    # id
    # title_id = ['zjnsrsbh']
    # print(len(get_repeat_content(feature_df['zjnsrsbh'])))
    # 类别基本不同：经营范围 8999/10000=0.8999
    # title_category = ['jyfw']
    # 地址基本相同：街道乡镇 9996/10000=0.9996
    # title_addr = ['jdxz']
    
    # 数值预处理：object转float64/int64,正则化
    feature_df[title_float64+title_int64] = feature_df[title_float64+title_int64].astype(np.float64)
    # feature_df[title_int64] = feature_df[title_int64].astype(np.int64)
    if with_norm:
        for title in title_float64 + title_int64:
            value = feature_df[title]
            value = (value - value.mean()) / value.std()
            feature_df.loc[:, title] = value
    feature_titles.extend(title_float64)
    feature_titles.extend(title_int64)

    # 类别预处理：object转onehot
    onehot_dfs = list()
    for title in title_category:
        # all_len = len(feature_df[title])
        # type_len = len(get_type2index(feature_df[title]))
        # print(f"{title}\t{type_len}/{all_len}={type_len/all_len}")
        onehot_df = get_onehot(feature_df, title)
        onehot_dfs.append(onehot_df)
        feature_titles.extend(onehot_df.columns)
    feature_df = pd.concat([feature_df] + onehot_dfs, axis=1)
    
    # 日期预处理：注册日期减2年，最后开票日期减注册日期
    # title_date = ['djkyrq']
    interval_title = 'interval_session'
    feature_df[interval_title] = feature_df.apply(get_interval_session, axis=1)
    value = feature_df[interval_title]
    value = (value - value.mean()) / value.std()
    feature_df.loc[:, interval_title] = value
    feature_titles.append(interval_title)

    # 地址/电话号码/名称预处理：是否重复出现
    to_check_titles = title_addr + title_number + title_name
    for title in to_check_titles:
        repeat_title = title + "_is_repeat"
        # print(f"{title}\t{len(get_repeat_content(feature_df[title]))}")
        column = feature_df[title]
        repeat_content = get_repeat_content(column)
        feature_df[repeat_title] = column.apply(get_repeat_value, repeat_content = repeat_content)
        feature_titles.append(repeat_title)

    return feature_df, feature_titles

def create_submit_data(test_data, prob_list):
    prob = {
        "zjnsrsbh": test_data['zjnsrsbh'],
    }
    submit_data = pd.DataFrame(prob)
    #all_one_or_zero(prob_list)
    submit_data['Probability'] = prob_list
    return submit_data

In [8]:
import copy
train_data = train_data.sample(frac=1)
train_x, train_y = train_data.drop('yc', axis=1), train_data['yc'].values
feature_df = copy.deepcopy(pd.concat([train_x, test_data], ignore_index=True))
# feature_df = fillna_with_mean(feature_df)
feature_df.head(1)
features, fea_columns = preprocess_feature(feature_df)
# backup: 
invoice_columns = [x for x in fea_columns if 'hy' not in x and 'repeat' not in x and 'session' not in x]
#fea_columns = [x for x in fea_columns if 'hy' not in x and 'repeat' not in x]
invoice_features = features[invoice_columns]
hy_columns = [x for x in fea_columns if 'hy' in x]
hy_features = features[hy_columns]

print(invoice_features.shape, hy_features.shape, train_y.shape)
invoice_features = invoice_features.values.astype(float)
hy_features = hy_features.values.astype(float)

train_y = train_y.astype(float)
train_invoice, test_invoice = invoice_features[:20000,],invoice_features[20000:,]
train_hy, test_hy = hy_features[:20000,],hy_features[20000:,]
print(train_invoice.shape, test_invoice.shape)
print(train_hy.shape, test_hy.shape)

(26000, 97) (26000, 1043) (20000,)
(20000, 97) (6000, 97)
(20000, 1043) (6000, 1043)


# package install

In [9]:
!pip install tensorflow==1.14
!pip install xgboost

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


# DNN

In [10]:
class NNClassfier():
    def __init__(self, placeholders, learning_rate, weight_decay, use_hy=False):
        self.loss = 0
        self.accuracy = 0
        self.placeholders = placeholders
        self.weight_decay = weight_decay
        self.hidden_size = 100
        self.use_hy = use_hy
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.build()
        
    # def loss_func():
        
    def build(self):
        invoice_input = self.placeholders['invoice_input']
        invoice_out = tf.layers.dense(inputs=invoice_input, units=100, activation=tf.nn.relu)
        invoice_out = tf.layers.dense(inputs=invoice_out, units=100, activation=tf.nn.relu)
        
        hy_input = self.placeholders['hy_input']
        hy_out = tf.layers.dense(inputs=hy_input, units=100, activation=tf.nn.relu)
        hy_out = tf.layers.dense(inputs=hy_out, units=100, activation=tf.nn.relu)
        
        hidden_out = tf.concat([invoice_out, hy_out], axis=-1)
        hidden_out = tf.layers.dense(inputs=hidden_out, units=100, activation=tf.nn.relu)
        
        if not self.use_hy:
            hidden_out = invoice_out
        logits = tf.layers.dense(inputs=hidden_out, units=1, activation=None)
        self.scores = tf.nn.sigmoid(logits)
        
        varss = tf.trainable_variables()
        labels = self.placeholders['labels']
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
        #self.loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(labels, logits, 0.1, name=None))
        self.loss += tf.add_n([tf.nn.l2_loss(v) for v in varss]) * self.weight_decay
        
        self.opt_op = self.optimizer.minimize(self.loss)
        

def linear_model(X, y, class_weight={0: 0.1, 1: 0.9}):
    model = LogisticRegression(class_weight=class_weight, solver='lbfgs')
    model.fit(X, y)
    return model

model = linear_model(train_invoice, train_y)

In [15]:
class InvoiceDetection():
    
    def __init__(self, invoice_dim, hy_dim=1, company_dim=1, repeat_dim=1, batch_size=512, lr=0.001, lambda_=1e-4, epoch=40, dropout=0.5):
        
        self.lr = lr
        self.lambda_ = lambda_
        self.dropout = dropout
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.train_num = 0
        self.test_num = 0
        self.invoice_data = None
        self.hy_data = None
        self.label_data = None
        self.company_data = None
        self.repeat_data = None
        
        self.invoice_data_test = None
        self.hy_data_test = None
        self.label_data_test = None
        self.company_data_test = None
        self.repeat_data_test = None
        self.test_id = None
        
        self.placeholders = {
            "invoice_input": tf.placeholder(tf.float64, shape=(None, invoice_dim), name='invoice_input'),
            "company_input": tf.placeholder(tf.float64, shape=(None, company_dim), name='company_input'),
            "hy_input": tf.placeholder(tf.float64, shape=(None, hy_dim), name='hy_input'),
            "repeat_input": tf.placeholder(tf.float64, shape=(None, repeat_dim), name='repeat_input'),
            "labels": tf.placeholder(tf.float64, shape=(None, 1), name='labels')
        }
        
        self.model = NNClassfier(self.placeholders, self.lr, self.lambda_)
        
        # Initialize session
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
        self.config = tf.ConfigProto(gpu_options=gpu_options)
        self.config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=self.config)
        
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        
        
    def construct_feed_dict(self, invoice_batch, label_batch, test_mode=False, hy_batch=[], company_batch=[], repeat_batch=[]):
        feed_dict = dict()
        if not test_mode:
            feed_dict.update({self.placeholders['labels']: label_batch})
            
        feed_dict.update({self.placeholders['invoice_input']: invoice_batch})
        
        if len(hy_batch) != 0:
            feed_dict.update({self.placeholders['hy_input']: hy_batch})
        if len(company_batch) != 0:
            feed_dict.update({self.placeholders['company_input']: company_batch})
        if len(repeat_batch) != 0:
            feed_dict.update({self.placeholders['repeat_input']: repeat_batch})
            
        return feed_dict
        
    def load_data(self, train_, test_):
        self.invoice_data, self.hy_data, self.label_data = train_
        self.test_id, self.invoice_data_test, self.hy_data_test = test_
        #self.invoice_data, self.label_data = train_
        #self.test_id, self.invoice_data_test = test_
        self.train_num = self.label_data.shape[0]
        self.test_num = self.test_id.shape[0]

    # invoice_data, hy_data, label_data, company_data=None, repeat_data=None
    def train(self):
        # start...
        for epo in range(self.epoch):
            losses, count = 0, 0
            for i in range(0, self.train_num, self.batch_size):
                feed_dict = self.construct_feed_dict(self.invoice_data[i:i+self.batch_size, :],
                                                     self.label_data[i:i+self.batch_size, :],
                                                     hy_batch=self.hy_data[i:i+self.batch_size, :])
                #print(feed_dict)
                _, loss = self.sess.run([self.model.opt_op, self.model.loss], feed_dict=feed_dict)
                losses += loss
                count += 1
            # print("EPOCH", epo, " | LOSS", loss, " | ACC", accuracy)
            print("EPOCH", epo, "| LOSS", loss)
            if epo % 5 == 0:
                self.test()
            # print("EPOCH", epo, " | Train Loss", losses / count, " | Test Loss", loss_test, " | ACC", acc)
            
    def test(self):
        losses, count, predict_proba = 0, 0, []
        for i in range(0, self.test_num, self.batch_size):
            feed_dict = self.construct_feed_dict(self.invoice_data_test[i:i+self.batch_size, :], None,
                                                 test_mode=True,
                                                 hy_batch=self.hy_data_test[i:i+self.batch_size, :]
                                                 )
            # print(feed_dict)
            probs = self.sess.run(self.model.scores, feed_dict=feed_dict)
            #losses += loss
            count += 1
            predict_proba += probs.T[0].tolist()
        
#         get_score.post_user_id('10')
#         predict_proba = np.array(predict_proba)
#         print(predict_proba[:10])
#         submit_data = create_submit_data(self.test_id, predict_proba)
#         get_score.post_verify_data(submit_data)

In [16]:
import tensorflow as tf
tf.__version__

processor = InvoiceDetection(train_invoice.shape[1], train_hy.shape[1], lambda_=0.0001)

train_ = (train_invoice, train_hy, train_y.reshape(-1,1))
test_ = (test_data[['zjnsrsbh']], test_invoice, test_hy)
processor.load_data(train_, test_)

processor.train()

EPOCH 0 | LOSS 0.3059950571971946
EPOCH 1 | LOSS 0.21609742908266388
EPOCH 2 | LOSS 0.15229456156429227
EPOCH 3 | LOSS 0.12459314944224506
EPOCH 4 | LOSS 0.1170598972877252
EPOCH 5 | LOSS 0.11382046177682283
EPOCH 6 | LOSS 0.11219009040280115
EPOCH 7 | LOSS 0.11190093850523282
EPOCH 8 | LOSS 0.11329653273867446
EPOCH 9 | LOSS 0.11159888063521618
EPOCH 10 | LOSS 0.11270272668890532
EPOCH 11 | LOSS 0.1116691626684612
EPOCH 12 | LOSS 0.1100719408133644
EPOCH 13 | LOSS 0.1131694988882202
EPOCH 14 | LOSS 0.11204958270412763
EPOCH 15 | LOSS 0.11236799938345396
EPOCH 16 | LOSS 0.11140600802066145
EPOCH 17 | LOSS 0.11205063774511567
EPOCH 18 | LOSS 0.10915200079603622
EPOCH 19 | LOSS 0.11013590720935057
EPOCH 20 | LOSS 0.11274917205594953
EPOCH 21 | LOSS 0.10994386482809196
EPOCH 22 | LOSS 0.1126278281582967
EPOCH 23 | LOSS 0.10938084441750588
EPOCH 24 | LOSS 0.11179047847110904
EPOCH 25 | LOSS 0.10784722428880836
EPOCH 26 | LOSS 0.11252277576544978
EPOCH 27 | LOSS 0.11048199931041007
EPOCH 28

# Xgboost

In [18]:
import xgboost
xgb = xgboost.XGBClassifier(nthread=4,
                          learning_rate=0.08,
                          n_estimators=250,
                          max_depth=5,
                          gamma=0,
                          subsample=0.9,
                          colsample_bytree=0.5)

xgb.fit(train_invoice, train_y)

get_score.post_user_id('10')
predict_proba = xgb.predict_proba(test_invoice)[:, 1].astype(np.float64)
submit_data = create_submit_data(test_data, predict_proba)
# get_score.post_verify_data(submit_data)

# MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
# (0.001,0.001)
mlp = MLPClassifier(activation='relu',
                    solver='adam',
                    alpha=0.001, # 0.001, 0.01
                    learning_rate_init=0.001, # 0.05
                    max_iter=500,
                    learning_rate='invscaling') # constant, adaptive, invscaling
mlp.fit(train_invoice, train_y)

get_score.post_user_id('10')
predict_proba = mlp.predict_proba(test_invoice)[:, 1].astype(np.float64)
submit_data = create_submit_data(test_data, predict_proba)
# get_score.post_verify_data(submit_data)

# SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC(C=1.0, kernel='rbf', gamma='auto', probability=True)
svm.fit(train_invoice, train_y)

get_score.post_user_id('10')
predict_proba = svm.predict_proba(test_invoice)[:, 1].astype(np.float64)
submit_data = create_submit_data(test_data, predict_proba)
# get_score.post_verify_data(submit_data)

# bayes

In [17]:
from sklearn import naive_bayes
bayes = naive_bayes.BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)
bayes.fit(train_invoice, train_y)

get_score.post_user_id('10')
predict_proba = bayes.predict_proba(test_invoice, )[:, 1].astype(np.float64)
submit_data = create_submit_data(test_data, predict_proba)
# get_score.post_verify_data(submit_data)